In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob

path = r'.\Data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
frame.shape

(5355214, 25)

# Data Preprocessing

At  the moment we have decided to filter the busiest airline, thereafter look at the top-3 most used airports to departure and arrival from/to. 

In [3]:
busiest_airline_id = frame['OP_CARRIER_AIRLINE_ID'].value_counts().head(1).index[0]
busiest_airline_id

19393

In [4]:
airline_ids = pd.read_csv('airport_airline_data/L_AIRLINE_ID.csv')
airline_ids[airline_ids['Code']==busiest_airline_id]

,Code,Description
362,19393,Southwest Airlines Co.: WN


In [5]:
airports_filter = []
three_departure = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]['ORIGIN_AIRPORT_ID'].value_counts().head(3).index.to_list()
three_arrival = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]['DEST_AIRPORT_ID'].value_counts().head(3).index.to_list()
airports_filter.extend(x for x in three_departure if x not in airports_filter)
airports_filter.extend(x for x in three_arrival if x not in airports_filter)
airports_filter

[13232, 11292, 12889]

In [6]:
airport_ids = pd.read_csv('airport_airline_data/L_AIRPORT_ID.csv')
airport_ids[airport_ids.Code.isin(airports_filter)]

,Code,Description
1245,11292,"Denver, CO: Denver International"
2762,12889,"Las Vegas, NV: McCarran International"
3098,13232,"Chicago, IL: Chicago Midway International"


Filter df with busiest airline with top 3 departure, arrival airports

In [7]:
busiest_airline_df = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]
filtered_df = busiest_airline_df[(busiest_airline_df.DEST_AIRPORT_ID.isin(airports_filter)) | (busiest_airline_df.ORIGIN_AIRPORT_ID.isin(airports_filter))]
filtered_df

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
116754,2018,4,2,1,19393,13232,13871,2000,2026.0,26.0,...,90.0,87.0,1.0,423.0,2,0.0,0.0,0.0,0.0,23.0
116755,2018,4,2,1,19393,13232,13871,825,818.0,-7.0,...,90.0,88.0,1.0,423.0,2,NaN,NaN,NaN,NaN,NaN
116756,2018,4,2,1,19393,13232,13871,1015,1021.0,6.0,...,85.0,85.0,1.0,423.0,2,NaN,NaN,NaN,NaN,NaN
116757,2018,4,2,1,19393,13232,13891,2005,2005.0,0.0,...,260.0,252.0,1.0,1706.0,7,NaN,NaN,NaN,NaN,NaN
116758,2018,4,2,1,19393,13232,13931,2305,2312.0,7.0,...,110.0,102.0,1.0,704.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355194,2022,7,13,3,19393,11638,11292,1455,1458.0,3.0,...,135.0,133.0,1.0,844.0,4,NaN,NaN,NaN,NaN,NaN
5355195,2022,7,13,3,19393,11638,12889,1550,1544.0,-6.0,...,70.0,68.0,1.0,259.0,2,NaN,NaN,NaN,NaN,NaN
5355196,2022,7,13,3,19393,11638,12889,1115,1226.0,71.0,...,70.0,77.0,1.0,259.0,2,0.0,0.0,7.0,0.0,71.0
5355197,2022,7,13,3,19393,11638,12889,555,549.0,-6.0,...,65.0,65.0,1.0,259.0,2,NaN,NaN,NaN,NaN,NaN


In [8]:
# Number of flights that had delay due to late aircraft 
filtered_df[(filtered_df['LATE_AIRCRAFT_DELAY']>0)].shape

(55625, 25)

In [9]:
# Number of flights that had delay due to carrier 
filtered_df[(filtered_df['CARRIER_DELAY']>0)].shape

(58253, 25)

In [10]:
# Number of flights that had delay due to NAS 
filtered_df[filtered_df['NAS_DELAY']>0].shape

(32468, 25)

In [11]:
# Number of flights that had delay due to carrier and NAS 
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['NAS_DELAY']>0)].shape


(18328, 25)

In [12]:
# Number of flights that had delay due to carrier and late aircraft 
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['LATE_AIRCRAFT_DELAY']>0)].shape

(39512, 25)

In [13]:
# Number of flights that had delay due to NAS and late aircraft
filtered_df[(filtered_df['NAS_DELAY']>0)&(filtered_df['LATE_AIRCRAFT_DELAY']>0)].shape

(17208, 25)

In [14]:
# Number of flights that had delay due to carrier, NAS and late aircraft
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['LATE_AIRCRAFT_DELAY']>0)&(filtered_df['NAS_DELAY']>0)].shape

(11056, 25)

In [15]:
filtered_df[(filtered_df['CANCELLED']==1)|(filtered_df['DIVERTED']==1)]

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
117696,2018,4,19,4,19393,14771,12889,2145,NaN,NaN,...,90.0,NaN,1.0,414.0,2,NaN,NaN,NaN,NaN,NaN
119597,2018,4,23,1,19393,14107,12889,2025,NaN,NaN,...,65.0,NaN,1.0,255.0,2,NaN,NaN,NaN,NaN,NaN
119602,2018,4,23,1,19393,14107,12889,1640,NaN,NaN,...,60.0,NaN,1.0,255.0,2,NaN,NaN,NaN,NaN,NaN
119624,2018,4,23,1,19393,14107,13232,730,NaN,NaN,...,195.0,NaN,1.0,1444.0,6,NaN,NaN,NaN,NaN,NaN
119871,2018,4,23,1,19393,14679,11292,1600,1708.0,68.0,...,135.0,NaN,1.0,853.0,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352346,2022,7,6,3,19393,13232,15016,905,NaN,NaN,...,70.0,NaN,1.0,251.0,2,NaN,NaN,NaN,NaN,NaN
5352511,2022,7,6,3,19393,13796,11292,2140,NaN,NaN,...,140.0,NaN,1.0,957.0,4,NaN,NaN,NaN,NaN,NaN
5353325,2022,7,6,3,19393,14831,11292,2130,NaN,NaN,...,145.0,NaN,1.0,948.0,4,NaN,NaN,NaN,NaN,NaN
5353435,2022,7,6,3,19393,14869,11292,1655,NaN,NaN,...,85.0,NaN,1.0,391.0,2,NaN,NaN,NaN,NaN,NaN


In [16]:
filtered_df[['CRS_DEP_TIME','DEP_TIME','DEP_DELAY']]

,CRS_DEP_TIME,DEP_TIME,DEP_DELAY
116754,2000,2026.0,26.0
116755,825,818.0,-7.0
116756,1015,1021.0,6.0
116757,2005,2005.0,0.0
116758,2305,2312.0,7.0
...,...,...,...
5355194,1455,1458.0,3.0
5355195,1550,1544.0,-6.0
5355196,1115,1226.0,71.0
5355197,555,549.0,-6.0


In [17]:
filtered_df[['CRS_ARR_TIME','ARR_TIME','ARR_DELAY']]

,CRS_ARR_TIME,ARR_TIME,ARR_DELAY
116754,2130,2153.0,23.0
116755,955,946.0,-9.0
116756,1140,1146.0,6.0
116757,2225,2217.0,-8.0
116758,155,154.0,-1.0
...,...,...,...
5355194,1810,1811.0,1.0
5355195,1700,1652.0,-8.0
5355196,1225,1343.0,78.0
5355197,700,654.0,-6.0


In [18]:
filtered_df.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_AIRLINE_ID',
       'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'FLIGHTS',
       'DISTANCE', 'DISTANCE_GROUP', 'CARRIER_DELAY', 'WEATHER_DELAY',
       'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'],
      dtype='object')

### Delete cancelled and diverted flights

In [19]:
filtered_df = filtered_df[(filtered_df['CANCELLED']!=1)|(filtered_df['DIVERTED']!=1)]

### Select the relevant columns

In [20]:
inc_columns = ['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','OP_CARRIER_AIRLINE_ID','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','CRS_DEP_TIME','CRS_ARR_TIME','CRS_ELAPSED_TIME','DISTANCE','DISTANCE_GROUP','DEP_DELAY','ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']

In [21]:
#https://bestcarsanswers.com/library/article/read/32856-what-is-crs-elapsed-time-of-flight
filtered_df2 = filtered_df[inc_columns]
filtered_df2['DELAYED'] = (filtered_df2['DEP_DELAY']>=15)|(filtered_df2['ARR_DELAY']>=15)
filtered_df2.head(2)

c:\Users\tijnv\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,DEP_DELAY,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DELAYED
116754,2018,4,2,1,19393,13232,13871,2000,2130,90.0,423.0,2,26.0,23.0,0.0,0.0,0.0,0.0,23.0,True
116755,2018,4,2,1,19393,13232,13871,825,955,90.0,423.0,2,-7.0,-9.0,NaN,NaN,NaN,NaN,NaN,False


### EXTRA PREPROCESSING

In [22]:
def categories_cat_delay(series: pd.Series) -> list:
    lst = []
    for value in series:
        if value < 1:
            if value != 0:
                print(value)
            lst.append(0)
        elif value > 0: 
            lst.append('Greater than 0')
        elif pd.isna(value):
            lst.append('nan')
        else:
            print(value)
    return lst

In [23]:
filtered_df2['CARRIER_DELAY_BOOL'] = categories_cat_delay(filtered_df2['CARRIER_DELAY'])

c:\Users\tijnv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
#This output shows whether if a flight is delayed (has a departure or arrival delay >= 15), it has a value within a categorical delay column
#If there is a value in one of the categorical delay columns all categorical delay columns have a value <- True
#Therefore this output only shows it for carrier_delay column too make it more easy to read
filtered_df2[['DELAYED','CARRIER_DELAY_BOOL']].value_counts()


DELAYED  CARRIER_DELAY_BOOL
False    nan                   292293
True     Greater than 0         58253
         0                      24619
         nan                    24562
dtype: int64

In [25]:
# ^ There are 24562 rows that are delayed but do not have values for the reasoning behind this (values in the categorical delay columns)
# Therefore delete these rows! Do not have information about the categorical delay columns eventhough these flights are delayed.

In [26]:
filtered_df3 = filtered_df2[~((filtered_df2['DELAYED']==True) & (filtered_df2['CARRIER_DELAY_BOOL'] == 'nan'))]

In [27]:
df_3 = filtered_df3[['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY','DELAYED']].copy()

In [28]:
df_3['CARRIER_DELAY_BOOL'] = categories_cat_delay(df_3['CARRIER_DELAY'])
df_3['WEATHER_DELAY_BOOL'] = categories_cat_delay(df_3['WEATHER_DELAY'])
df_3['NAS_DELAY_BOOL'] = categories_cat_delay(df_3['NAS_DELAY'])
df_3['SECURITY_DELAY_BOOL'] = categories_cat_delay(df_3['SECURITY_DELAY'])
df_3['LATE_AIRCRAFT_DELAY_BOOL'] = categories_cat_delay(df_3['LATE_AIRCRAFT_DELAY'])


In [29]:
df_3[['DELAYED','CARRIER_DELAY_BOOL','WEATHER_DELAY_BOOL','NAS_DELAY_BOOL','SECURITY_DELAY_BOOL','LATE_AIRCRAFT_DELAY_BOOL']].value_counts()

DELAYED  CARRIER_DELAY_BOOL  WEATHER_DELAY_BOOL  NAS_DELAY_BOOL  SECURITY_DELAY_BOOL  LATE_AIRCRAFT_DELAY_BOOL
False    nan                 nan                 nan             nan                  nan                         292293
True     Greater than 0      0                   0               0                    Greater than 0               28456
                                                                                      0                            11469
                                                 Greater than 0  0                    Greater than 0               11056
         0                   0                   0               0                    Greater than 0                9289
                                                 Greater than 0  0                    0                             7506
         Greater than 0      0                   Greater than 0  0                    0                             7272
         0                   0            

In [30]:
filtered_df4 = filtered_df3[inc_columns]

In [31]:
filtered_df5 = filtered_df4.loc[:, ~filtered_df4.columns.isin(['DEP_DELAY', 'ARR_DELAY'])].fillna(0)

## Carrier
Carrier df including flights with and without delay, X are variables that are known beforehand about the flight and Y is the delay in minutes due to the carrier

In [32]:
carrier_df = filtered_df5[['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','OP_CARRIER_AIRLINE_ID','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','CRS_DEP_TIME','CRS_ARR_TIME','CRS_ELAPSED_TIME','DISTANCE','DISTANCE_GROUP','CARRIER_DELAY']].copy()

In [33]:
X = carrier_df[['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','OP_CARRIER_AIRLINE_ID','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','CRS_DEP_TIME','CRS_ARR_TIME','CRS_ELAPSED_TIME','DISTANCE','DISTANCE_GROUP']].copy()
X

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP
116754,2018,4,2,1,19393,13232,13871,2000,2130,90.0,423.0,2
116755,2018,4,2,1,19393,13232,13871,825,955,90.0,423.0,2
116756,2018,4,2,1,19393,13232,13871,1015,1140,85.0,423.0,2
116757,2018,4,2,1,19393,13232,13891,2005,2225,260.0,1706.0,7
116758,2018,4,2,1,19393,13232,13931,2305,155,110.0,704.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
5355194,2022,7,13,3,19393,11638,11292,1455,1810,135.0,844.0,4
5355195,2022,7,13,3,19393,11638,12889,1550,1700,70.0,259.0,2
5355196,2022,7,13,3,19393,11638,12889,1115,1225,70.0,259.0,2
5355197,2022,7,13,3,19393,11638,12889,555,700,65.0,259.0,2


In [34]:
y = carrier_df[['CARRIER_DELAY']]
y

,CARRIER_DELAY
116754,0.0
116755,0.0
116756,0.0
116757,0.0
116758,0.0
...,...
5355194,0.0
5355195,0.0
5355196,0.0
5355197,0.0
